<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png" width="40px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Decisions with Semantic Functions</span>
</div>

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks with their descriptions 
* Listing types of decision-making frameworks dynamically based on the type  

Semantic Kernel Functions are the core building blocks of functionality for intelligent AI orchestration. Semantic Kernel functions usually have a single responsibility to perform. For example, a Sementic Kernel function can: send an e-mail, call an API, recommend a reasoning framework for a high-stakes decision etc. 

What makes Semantic Kernel functions so special? Can't one just produce similar outcomes by using a prompt in an LLM or just writing a software program? In order to answer that, one has to look at what makes GenAI so innovative. The GenAI models have a unique ability to process instructions with reasoning and logic. This allows these models to behave almost like a human decision maker. Even with basic prompts, GenAI models perform reasonably well. However, providing GenAI models with additional functions allows them to gain access to business processes, data and basically anything a GenAI model should consider when performing AI orchestration. 

For example, imagine you want to prepare a great Thanksgiving dinner and want to get help from a GenAI cooking application to create a new recipe. You enter what you want todo and it comes up with the most delicious looking feast for Thanksgiving. However, there is a problem it used ingredients and recommended using cooking appliances that you do not own! You could enter all of the ingredients and keep prompting until it narrowed down the what you could realistically make. Wouldn't it be better if the GenAI cooking application had access to: your available ingredients, your time availability, kitchen appliances and even your allergic preferences. Now the GenAI model can craft a Thanksgiving feast understanding the parameters and data without having to be guided. This is exactly what Semantic Kernel functions provide.

Semantic Kernel functions come in two flavors: 
* Semantic functions
* Native functions

Semantic functions are basically **prompt instructions** that help guide GenAI LLMs around the specific guidelines it should adhere to when building the AI orchestration. Therefore, semantic functions are likely the first place most AI architects/engineers start when composing AI orchestration with Semantic Kernel. Semantic functions can also take in parameters for dynamic creaation of instructions.  

Below is an example of semantic function that can be used for facilitating decision-making. Note the **{{$decisionToMake}}** input where the parameter can be dynamically passed in to provide additional specificity.

```javascript
[DECISION FRAMEWORKS TO USE]
Price's Law
Pareto Principle
Laplace Rule of Succession
Eisenhower Matrix
Median Rule of Five
Second Order Thinking

[DECISION GUIDANCE]
Try to use the best fitting framework.  
Prefer using quantitative decision frameworks rather qualitative ones.  
Use the code interpreter to validate quantitative reasoning.

Use the decision frameworks above to help the user make the following decision:
{{$decisionToMake}}
```  

The image below illustrates the core value of semantic functions. 

<img style="display: block; margin: auto;" width ="500px" src="https://learn.microsoft.com/en-us/semantic-kernel/media/semantic-function-explainer.png">

----
### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 9.0.6"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.6"
#r "nuget: Microsoft.SemanticKernel, 1.60.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.60.0-preview"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.6 Microsoft.Extensions.Configuration.Json, 9.0.6 Microsoft.SemanticKernel, 1.60.0 Microsoft.SemanticKernel.Plugins.Core, 1.60.0-preview

In [2]:
Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Using Azure OpenAI Service


----
### Step 2 - Create a Semantic Kernel - Semantic Function 

> 📜 **_"Decision frameworks are like maps. Use them to navigate complex decision-making terrain._** 
>
> -- <cite>Unknown</cite> 

Semantic functions are constructs inside Semantic Kernel that are basically just prompt instructions. What makes the special is that they can be exposed as functions, thus it makes re-usability easy. Furthermore, future modules will show how they can faciliate composability into plugins. 

In [10]:
// Simple prompt to list some decision frameworks this GenAI LLM is familiar with
var decisionPromptTemplate = """
Identify and list various decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each framework supports better analysis and reasoning in different decision-making scenarios.

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";

// Takes a regular prompt and creats a Semantick Kernel function that can be used to invoke the model
// The prompt is now a Semantic Kernel Function, which can be treated as a first-class object
var semanticDecisionFunction = semanticKernel.CreateFunctionFromPrompt(decisionPromptTemplate);

// Directly invoke the Semantic Kerenel Function with the decision prompt built-in
var decisionFunctionResponse = await semanticKernel.InvokeAsync(semanticDecisionFunction);

var decisionFunctionResponseString = decisionFunctionResponse.GetValue<string>(); 
decisionFunctionResponseString.DisplayAs("text/markdown");

| ### Framework Name          | ### Description & Usage Scenario                                                                                                    |
|----------------------------|---------------------------------------------------------------------------------------------------------------------------------------|
| Rational Decision-Making Model | Breaks down decisions into logical steps: define problem, identify criteria, weigh options, evaluate alternatives, choose solution. Ideal for complex, high-stakes decisions where data is available.                |
| SWOT Analysis              | Assesses Strengths, Weaknesses, Opportunities, and Threats to an option or organization, promoting a balanced view for strategic planning and risk assessment.                  |
| Cost-Benefit Analysis      | Weighs costs and benefits (quantitative and qualitative) of alternatives, supporting choices where efficiency and value are primary concerns (e.g., project selection).          |
| Pros and Cons List         | Lists positive and negative aspects, making reasoning explicit and helping clarify preferences for personal or group decisions under moderate complexity.                      |
| Pareto Analysis            | Focuses on identifying a small number of causes that lead to a majority of problems (80/20 rule), guiding teams to prioritize impactful solutions.                            |
| Six Thinking Hats          | Encourages looking at problems from multiple perspectives (logic, emotion, caution, optimism, creativity, overview), improving creativity and reducing groupthink.            |
| OODA Loop                  | Stands for Observe, Orient, Decide, Act. Emphasizes adapting quickly to changing situations, useful for fast-paced or competitive environments.                                |
| Decision Matrix (Grid Analysis)| Ranks options against weighted criteria, supporting objective analysis in multi-factor decisions (e.g., hiring, vendor selection).                                           |
| Delphi Technique           | Iterative aggregation of expert opinions, reducing bias and encouraging consensus, ideal for forecasting or future-oriented group decisions.                                   |
| The Eisenhower Matrix      | Categorizes tasks by urgency and importance (four quadrants) to support prioritization decisions in time management and resource allocation.                                    |
| Nudge Theory               | Designs decision environments to encourage better choices without restricting options, beneficial in behavioral economics and policy settings.                                 |
| 5 Whys Analysis            | Repeatedly asks "why" to drill down to the root cause of a problem, enhancing problem-solving decisions by addressing underlying issues.                                       |
| Pre-mortem Analysis        | Instructs teams to imagine a future failure and work backwards, proactively identifying risks and weaknesses in a proposed plan for better risk mitigation.                    |

----
### Step 3 - Semantic Function Dynamic Decision Intelligence  

> 📜 **_"If the only tool you have is a hammer, you tend to see every problem as a nail."_**
>
> -- <cite>Abraham Maslow (Renowned American psychologist)</cite> 

Semantic functions can be dynamically composed using parameter placeholders, which can then be dynamically rendered using Kernel arguments. This allows the ease of passing in parameters and execution settings into the semantic functions. This is not groundbreaking, but it does allow you to dynamically compose AI prompts (context-engineering, prompt-engineering dynamically).

Execute the cell below to view how the prompt can dynamically instruct the LLM to retrieve different types of decision-making frameworks. 

In [ ]:
// Takes a prompt with input variables and creates a function that can be used to invoke the GenAI model
var decisionPromptTemplate = """
Identify and list {{$numberOfFrameworks}} decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each framework supports better analysis and reasoning in {{$decisionType}} decision-making scenarios.

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";

// Create the OpenAI prompt execution settings, try changing the settings to see how the output changes
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var semanticDecisionFunctionWithInput = semanticKernel.CreateFunctionFromPrompt(decisionPromptTemplate, openAIPromptExecutionSettings);

// Dynamically set the number of frameworks and decision type -> strategic
var kernelArguments = new KernelArguments();
kernelArguments.Add("numberOfFrameworks", 3);
kernelArguments.Add("decisionType", "Long-Term strategic with probabilistic outcomes");

// Invoke the Semantic Kernel Function with the decision prompt built-in and the dynamic arguments
var decisionDynamicFunctionResponse = await semanticKernel.InvokeAsync(semanticDecisionFunctionWithInput, kernelArguments);

var decisionDynamicFunctionResponseString = decisionDynamicFunctionResponse.GetValue<string>(); 
decisionDynamicFunctionResponseString.DisplayAs("text/markdown");

| Framework Name                | Description of Framework                                                                 | How it Enhances Long-Term, Probabilistic Strategic Decision-Making                          |
|-------------------------------|----------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------|
| Decision Tree Analysis        | A graphical representation of possible choices, chance events, outcomes, and payoffs.  | Helps visualize and quantify complex, multi-stage decisions with uncertain outcomes, allowing for explicit calculation of expected values and identification of optimal strategies over time. |
| Monte Carlo Simulation        | Uses computational algorithms to simulate a wide range of possible outcomes based on probability distributions. | Enables robust analysis of uncertainty by modeling thousands of scenarios, providing insights into the range and likelihood of long-term strategic results.           |
| Real Options Analysis         | Treats strategic decisions as financial options, valuing flexibility and future choices under uncertainty. | Supports better reasoning by quantifying the value of waiting, expanding, or abandoning projects as new information emerges, aligning with dynamic, probabilistic environments. |

In the example below, the number of frameworks to return has been changed as well as the type has been changed to **Quick to Implement for rapid Dicision-Making"**.  

In [14]:
// Dynamically set the number of frameworks and decision type -> requring fast action
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings);
kernelArguments.Add("numberOfFrameworks", 5);
kernelArguments.Add("decisionType", "Quick to Implement for rapid Dicision-Making");

// Invoke the Semantic Kernel Function with the decision prompt built-in and the dynamic arguments
// Note: The invocation has NOT changed, only the Kernel arguments have changed
var decisionDynamicFunctionResponse = await semanticKernel.InvokeAsync(semanticDecisionFunctionWithInput, kernelArguments);

var decisionDynamicFunctionResponseString = decisionDynamicFunctionResponse.GetValue<string>(); 
decisionDynamicFunctionResponseString.DisplayAs("text/markdown");

| Framework Name         | Description & Support for Rapid Decision-Making                                                                                                                                         |
|------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| OODA Loop              | Stands for Observe, Orient, Decide, Act. This iterative cycle enables quick assessment of the situation, rapid orientation to context, fast decision, and immediate action—ideal for dynamic scenarios. |
| Eisenhower Matrix      | Prioritizes tasks by urgency and importance, helping decision-makers quickly focus on what matters most and avoid wasting time on less critical issues.                                   |
| 2x2 Decision Matrix    | Plots options on two axes (e.g., impact vs. effort), allowing for swift visual comparison and selection of the most favorable option in time-sensitive situations.                        |
| Pareto Analysis (80/20)| Identifies the vital few causes that contribute to the majority of results, enabling rapid focus on high-impact actions and efficient allocation of limited resources.                   |
| RAPID Framework        | Clarifies roles in decision-making (Recommend, Agree, Perform, Input, Decide), streamlining the process and reducing delays by ensuring everyone knows their responsibility.              |

----
### Step 4 - Decision Scenario with Dynamic Decision Intelligence

In the below code a decsion-analysis scenario is introduced that uses dynamic inputs to personalize the decision recommendation dynamically. The more specific information that provides contextual information to the Generative AI model can great improve the specific recommendations. 

**Decision Scenario:** Michael is a 35-year-old professional chef who is considering opening his own restaurant. This is a significant life decision that could greatly impact his career and personal life. Michael is looking for a recommendation for an approach (decision) for this potentially life-changing decision. 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-RestaurantDecision.png">

Three factors will be considered for this decision scenario that the user will be prompted for: 
1) Michael's Total Net Worth in dollars (enter number)
2) Level of competition with other restaurants (low, medium, high) 
3) Level of support from Michael's friends and family (low, medium, high) 

**Various Decision Inputs:** You can simulate different What-If scenarios by varying the input of net worth, restaurant competition and level of family support. Note the different decision recommendations based on this provided by Artificial Intelligence. The recommendations in this example are quite simple, but even on the extreme inputs Generative AI has a conecept of understanding the quality of inputs. 

In [18]:
// Import the Microsoft.DotNet.Interactive namespace for user input
using Microsoft.DotNet.Interactive;

var totalNetWorth = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Micheal's total net worth in dollars: ");
var levelOfCompetition = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Level of competition with other restaurants (Low, Medium, High): ");
var levelOfFamilySupport = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Level of support from Michael's friends and family (Low, Medium, High): ");

Console.WriteLine($"Michael's Net Worth: {totalNetWorth}");
Console.WriteLine($"Level of Restaurant Competition: {levelOfCompetition}");
Console.WriteLine($"Michael's level of Family Support: {levelOfFamilySupport}");
Console.WriteLine();

// Takes a prompt with input variables and creates a function that can be used to invoke the model
var restaurantDecisionRecommendation = """
Michael is a 35-year-old professional chef who is considering opening his own restaurant. 
This is a significant life decision that could greatly impact his career and personal life. 
Michael is looking for a recommendation on how to approach this.
Some key information about Michael:
- Michael's total net worth is (in US dollars) ${{$totalNetWorth}}.
- The level of competition with other restaurants is {{$levelOfCompetition}}.
- The level of support from Michael's friends and family is {{$levelOfFamilySupport}}.

Based on this information, what recommendation would you give to Michael regarding opening his own restaurant?

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
Provide a small Decision Summary of the recommendation below the table.
""";

// Create the OpenAI prompt execution settings, try changing the settings to see how the output changes
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1500, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var semanticRestaurantDecisionRecommendation = semanticKernel.CreateFunctionFromPrompt(restaurantDecisionRecommendation, openAIPromptExecutionSettings);

// Dynamically set the number of frameworks and decision type -> strategic
var kernelArgumentsRestaurantDecisionRecommendation = new KernelArguments();
kernelArgumentsRestaurantDecisionRecommendation.Add("totalNetWorth", totalNetWorth);
kernelArgumentsRestaurantDecisionRecommendation.Add("levelOfCompetition", levelOfCompetition);
kernelArgumentsRestaurantDecisionRecommendation.Add("levelOfFamilySupport", levelOfFamilySupport);

var decisionRestaurantDecisionRecommendationResponse = await semanticKernel.InvokeAsync(semanticRestaurantDecisionRecommendation, kernelArgumentsRestaurantDecisionRecommendation);

var decisionRestaurantDecisionRecommendationResponseString = decisionRestaurantDecisionRecommendationResponse.GetValue<string>(); 
decisionRestaurantDecisionRecommendationResponseString.DisplayAs("text/markdown");


Michael's Net Worth: 10000
Level of Restaurant Competition: Low
Michael's level of Family Support: Low



| Factor                          | Details                                                                 | Impact on Decision                         |
|----------------------------------|-------------------------------------------------------------------------|--------------------------------------------|
| Financial Resources              | Net worth: $10,000                                                      | Very limited capital; high risk            |
| Market Competition               | Low competition                                                          | Positive; more opportunity                 |
| Support System                   | Low support from friends and family                                      | Negative; may face emotional challenges    |
| Industry Experience              | Professional chef                                                        | Positive; relevant skills                  |
| Business Risk                    | High (due to low capital and support)                                    | Significant risk of failure                |
| Alternative Options              | Could consider partnerships, investors, or smaller-scale ventures first  | May reduce risk and increase support       |

  
Decision Summary:  
Given Michael’s limited financial resources and low personal support, opening a full-scale restaurant at this time is very risky, despite the favorable market conditions and his professional experience. It is recommended that Michael consider alternative approaches such as seeking investors, starting with a smaller venture (e.g., pop-up, food truck), or building a stronger support network before committing to a full restaurant launch.